In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dataPath = '/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/'

In [1]:
!pip install jsonlines

In [2]:
import huggingface_hub
huggingface_hub.login()

In [3]:
import pandas as pd
import json

In [4]:
import jsonlines

In [5]:
!pip install datasets==2.19.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.3.1 which is incompatible.


In [6]:
!pip install datasets

In [7]:
from datasets import Dataset

In [8]:
!pip install gspread gspread_dataframe


In [9]:
#데이터 경로 및 설정 불러오기
datasetName = "data.csv"
jsonFileName = "data.jsonl"

In [13]:
import pandas as pd
import json

def csv_to_json(csv_file_path, json_file_path):
    """Converts a CSV file to a JSONL file.

    Args:
        csv_file_path (str): The path to the CSV file.
        json_file_path (str): The path to the JSONL file.
    """
    try:
        # Try reading with 'utf-8' first
        df = pd.read_csv(csv_file_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            # If 'utf-8' fails, try 'cp949' (Korean)
            df = pd.read_csv(csv_file_path, encoding='cp949')
        except UnicodeDecodeError:
            try:
                # If 'cp949' fails, try 'latin-1' (Western European)
                df = pd.read_csv(csv_file_path, encoding='latin-1')
            except UnicodeDecodeError:
                # If 'latin-1' fails, try 'utf-16'
                df = pd.read_csv(csv_file_path, encoding='utf-16')

    # Proceed with writing to JSONL
    with open(json_file_path, "w", encoding='utf-8') as json_file:
        for index, row in df.iterrows():
            data = {'input': row['input'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write("\n")

# Example usage:
csv_file_path = 'data.csv'
json_file_path = 'data.jsonl'
csv_to_json(csv_file_path, json_file_path)


In [ ]:
!pip install gspread gspread_dataframe


**파인튜닝용 포맷변환(json)**

In [16]:
import jsonlines
from datasets import Dataset

# JSONLines 파일을 읽어서 데이터셋 생성
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
        indataset.append(f"<s>[INST] {line['input']} [/INST] {line['response']} </s>")

print('데이터셋 확인')
print(indataset[:5])

# 데이터셋을 사전으로 변환하여 Hugging Face Dataset으로 저장
indataset = Dataset.from_dict({'text': indataset})
indataset.save_to_disk('')

print('데이터셋 info 확인')
print(indataset)


데이터셋 확인
['<s>[INST] 2024년 12월 04일에 코레일, 총파업, 비상수송, 체제, 철도노조, 열차운행  와 관련된 뉴스의 내용은? [/INST] <b>한국철도공사</b>(코레일)가 5일로 예정된 철도노조의 총파업에 대비해 비상 수송 체제에 돌입했다. 4일 코레일에 따르면 비상 수송 대책본부를 24시간 운영하며 열차 운행 조정과 안전대책 수립 등 파업 대비 종합 대책을...  </s>', '<s>[INST] 2024년 12월 04일에 컨소시엄, 샬롬엔지니어링, 궤도안전 와 관련된 뉴스의 내용은? [/INST] <b>한국철도공사</b>와 컨소시엄을 통해 샬롬엔지니어링은 궤도 안전 분야의 독보적인 전문성을 바탕으로 궤도 안전 컨설팅을 시행하고 기자재 보급, 유지 보수 개량, 거점센터 구축, 전문인력 양성, 가이드라인 제작 및...  </s>', '<s>[INST] 2024년 12월 04일에 박상우, 국토교통부, 철도파업, 대비, 수송대책, 경찰청, 고용노동부  와 관련된 뉴스의 내용은? [/INST] 박상우 국토교통부 장관이 4일 오후 행정안전부, 고용노동부, 경찰청, 서울시, 인천시, 경기도, <b>한국철도공사</b>(코레일) 등 관계기관과 함께 코레일 노사 간 협상이 결렬될 경우를 대비해 철도파업 대비 수송대책 점검회의...  </s>', '<s>[INST] 2024년 12월 04일에 노조, 파업, 수인분당선, 경강선, 서울도시철도 와 관련된 뉴스의 내용은? [/INST] 시는 두 노조의 파업으로 <b>한국철도공사</b>(코레일)가 운영하는 수인분당선(복정역~오리역)과 경강선(판교역~이매역), 서울도시철도 8호선(복정역~모란역) 운행에 차질이 불가피할 것으로 보고 있다. 파업이 예정대로...  </s>', '<s>[INST] 2024년 12월 04일에 기본급, 인상, 철도, 총파업, 오는, 교대  와 관련된 뉴스의 내용은? [/INST] 앞서 철도노조는 오는 5일 철도 총파업을 예고하며 <b>한국철도공사</b>(코레일)에 △기본급 2.5% 인상 

Saving the dataset (0/1 shards):   0%|          | 0/151 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 151
})


In [17]:
indataset

Dataset({
    features: ['text'],
    num_rows: 151
})

In [18]:
# 허깅페이스에 업로드
indataset.push_to_hub("hwnam1129/llama3_railcar_trend")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hwnam1129/llama3_railcar_trend/commit/5908786d8cdd4f897e8cab4607baf3cf5170b320', commit_message='Upload dataset', commit_description='', oid='5908786d8cdd4f897e8cab4607baf3cf5170b320', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hwnam1129/llama3_railcar_trend', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hwnam1129/llama3_railcar_trend'), pr_revision=None, pr_num=None)